In [1]:
import pandas as pd
import requests
from IPython.display import clear_output
import time

In [17]:
final_df = pd.read_csv('데마프로젝트최종데이터.csv',index_col =0)
final_df.head()

,MATCH_ID,T1_top,T1_jug,T1_mid,T1_adc,T1_sup,T2_top,T2_jug,T2_mid,T2_adc,...,jug_counter,mid_counter,adc_counter,sup_counter,win,FB,FT,FI,FD,FJ
0,4404609027,4,1,5,2,5,2,2,4,1,...,50.53,28.99,47.61,37.90,Fail,0,0,0,0,1
1,4385275912,2,2,3,4,1,3,1,2,3,...,47.63,39.85,48.56,60.27,Win,1,0,1,1,0
2,4406640659,3,2,2,3,4,1,2,2,3,...,50.33,25.99,44.57,52.69,Fail,0,0,0,0,0
3,4385800216,2,4,5,4,4,2,2,4,1,...,48.60,33.81,52.32,47.00,Fail,0,0,0,1,0
4,4406640667,4,4,3,1,3,5,1,3,4,...,50.11,49.51,53.82,48.41,Win,1,1,1,1,1


In [20]:
for i in [1,2,3,4,5,6,7,8,9,10,17,18,19,20,21]:
    final_df[final_df.columns[i]] = final_df[final_df.columns[i]].astype('str') 

In [155]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz


In [21]:
final_df.dtypes

MATCH_ID         int64
T1_top          object
T1_jug          object
T1_mid          object
T1_adc          object
T1_sup          object
T2_top          object
T2_jug          object
T2_mid          object
T2_adc          object
T2_sup          object
top_counter    float64
jug_counter    float64
mid_counter    float64
adc_counter    float64
sup_counter    float64
win             object
FB              object
FT              object
FI              object
FD              object
FJ              object
dtype: object

In [8]:
class_df = final_df['win']

In [158]:
# 인게임 데이터
# 인게임 요소 기준 FI(첫 억제기 파괴) 제외
train_pre = final_df.loc[:,['FB','FT','FD','FJ' ]]

In [159]:
X_train, X_test, y_train, y_test = train_test_split(train_pre, class_df, stratify = class_df, random_state=42,train_size = 0.25)

In [160]:
#decision Tree
DT_accuracy_list =[]
DT_model_list = []
for depth in range(1,8):
    tree = DecisionTreeClassifier(max_depth = depth, random_state=123)
    tree.fit(X_train, y_train)

    y_pred = tree.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    DT_accuracy_list.append(accuracy)
    DT_model_list.append(tree)


best_index = DT_accuracy_list.index(max(DT_accuracy_list))+1
print("테스트 정확도: {:.3f}".format(max(DT_accuracy_list)))
print('최대 깊이 : {:d}'.format(best_index))
best_DT = DT_model_list[best_index-1]

테스트 정확도: 0.782
최대 깊이 : 2


In [161]:
best_DT.feature_importances_

array([0.37317835, 0.62682165, 0.        , 0.        ])

In [162]:
importance = best_DT.feature_importances_
importance_list = list(zip(X_train.columns, importance))

sorted(importance_list, key =lambda x:x[1], reverse=True)

[('FT', 0.6268216458133258),
 ('FB', 0.3731783541866742),
 ('FD', 0.0),
 ('FJ', 0.0)]

In [163]:
# random forest
RF_accuracy_list = []
RF_model_list =[]
for i in range(10):
    clf_rf = RandomForestClassifier()
    clf_rf.fit(X_train, y_train)
    y_pred = clf_rf.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    RF_accuracy_list.append(accuracy)
    RF_model_list.append(clf_rf)
    
print("테스트 정확도: {:.3f}".format(max(RF_accuracy_list)))
print(f'가장 성능이 좋은 모델: {(DT_accuracy_list.index(max(DT_accuracy_list)))+1}번째 모델')
best_RF = RF_model_list[(DT_accuracy_list.index(max(DT_accuracy_list)))]

테스트 정확도: 0.782
가장 성능이 좋은 모델: 2번째 모델


In [164]:
best_RF.feature_importances_

array([0.36960383, 0.4831102 , 0.09045516, 0.05683081])

In [165]:
importance2 = best_RF.feature_importances_
importance_list2 = list(zip(X_train.columns, importance2))

sorted(importance_list2, key =lambda x:x[1], reverse=True)

[('FT', 0.48311019605791516),
 ('FB', 0.36960382882787995),
 ('FD', 0.09045516208928221),
 ('FJ', 0.05683081302492275)]